In [1]:
from collections import defaultdict, Counter
import json
from operator import itemgetter
import os
import pickle
import spacy
from tqdm import tqdm

In [2]:
def generate_sequence(superspan_list, tokens):
    '''
        generate super span sequence
    '''
    index_in_token2index_in_superspan_list = [-1]*len(tokens)

    sequence = []
    for i, superspan in enumerate(superspan_list):
        st = superspan['st']
        ed = superspan['ed']
        for idx in range(st, ed):
            index_in_token2index_in_superspan_list[idx] = i
    
    index_in_superspan_list_that_has_occurred = set()
    for index_in_token, index_in_superspan_list in enumerate(index_in_token2index_in_superspan_list):
        if index_in_superspan_list == -1:
            # is plain text
            sequence.append({'tag': 'plain', 'text': tokens[index_in_token], 'st': index_in_token, 'ed': index_in_token + 1})
        elif index_in_superspan_list not in index_in_superspan_list_that_has_occurred:
            sequence.append(superspan_list[index_in_superspan_list])
            index_in_superspan_list_that_has_occurred.add(index_in_superspan_list)
            superspan_list[index_in_superspan_list].update({'tag': 'superspan'})

    return sequence

In [3]:
_nlp = spacy.load('en')

In [4]:
def filter_span_data_by_grammar(data, texts):
    new_data = []
    for doc_ind, (line, spans) in tqdm(enumerate(zip(texts, data)), total=len(texts)):
        if not spans:
            new_data.append(spans)
            continue
        text = str(line.strip())
        tokens = text.split(' ')
        try:
            doc = _nlp(text)
            pos_list = [token.pos_ for token in doc]
            assert len(tokens) == len(pos_list)
            new_spans = []
            for span in spans:
                if pos_list[span['ed']-1] not in ['NOUN', 'PROPN']:
                    log.info('doc_ind:{}, skipping {}'.format(doc_ind, span))
                    continue
                else:
                    ind = span['st']
                    while ind < span['ed']:
                        if pos_list[ind] not in ['NOUN', 'ADV', 'ADJ', 'PROPN']:
                            ind += 1
                        else:
                            break
                    if ind < span['ed']:
                        if span['st'] == ind:
                            new_span = span
                        else:
                            new_span = {'st': ind, 'ed': span['ed'], 'text': ' '.join(tokens[ind:span['ed']])}
                            log.info('modifying {} as {}'.format(span, new_span))
                        new_spans.append(new_span)
            new_data.append(new_spans)
        except Exception as e:
            print(e)
            new_data.append(spans)
            pass
    return new_data

In [5]:
def calc_phrase_freq(data, islower=True):
    try:
        source2freq_counter = defaultdict(Counter)
        for index, spans in tqdm(enumerate(data), total=len(data)):
            for span in spans:
                if islower:
                    token = span['text'].lower()
                else:
                    token = span['text']
                source2freq_counter[span['source']][token] += 1
        return source2freq_counter
    except Exception as e:
        print(e)

In [6]:
def merge_span_data(data_list, sourceNames=None):
    # check data length consistency
    length_list = [len(data) for data in data_list]
    print(length_list)
    assert len(set(length_list)) == 1

    length = length_list[0]
    new_data = []
    for i in range(length):
        new_d = []
        # todo: append sourcenames
        for data, source in zip(data_list, sourceNames):
            new_d.extend([dict(span, **{'source': source}) for span in data[i]])
        new_data.append(new_d)
    return new_data

In [7]:
def read_span_json(inFile, num=None):
    if num:
        with open(inFile) as fin:
            data = []
            for i, line in enumerate(fin):
                data.append(json.loads(line))
                if i >= num:
                    return data

    return [json.loads(line) for line in open(inFile)]

In [8]:
def read_text(inFile, num=None):
    texts = []
    with open(inFile) as f_text:
        if num:
            for cnt, line in enumerate(f_text):
                texts.append(line)
                if cnt > num:
                    break
        else:
            for line in f_text:
                texts.append(line)
        return texts

In [9]:
def validate_compress_superspan_sequence(superspan_sequence, tokens):
    for superspan in superspan_sequence:
        if superspan['tag'] == 'superspan':
            for span in superspan['spans']:
                assert span['st'] >= superspan['st']
                assert span['ed'] <= superspan['ed']
                assert span['text'] == ' '.join(tokens[span['st']:span['ed']])
    return superspan_sequence

In [10]:
def generate_superspan(d, tokens):
    cur_st = -1
    cur_ed = -1
    cur_superspan_ind = -1
    superspan = {}
    superspan_list = []
    for span in sorted(d, key=itemgetter('st', 'ed')):
        # span information
        st = span['st']
        ed = span['ed']
        text = span['text']
#       print('text', text)

        if st >= cur_ed:
            # meet a new superspan
            cur_st = st
            cur_ed = ed
            superspan = {'st': st,
                         'ed': ed, 'spans': []}
            superspan['spans'] = [span]
            superspan_list.append(superspan)
            cur_superspan_ind += 1
        else:
            # update super end pos
            cur_ed = max(ed, cur_ed)
            superspan_list[cur_superspan_ind]['ed'] = cur_ed
            if span not in superspan_list[cur_superspan_ind]['spans']:
                superspan_list[cur_superspan_ind]['spans'].append(span)
#             superspan_list[cur_superspan_ind]['text'] = ' '.join(tokens[cur_st:cur_ed])

    for i, superspan in enumerate(superspan_list):
        superspan_list[i]['text'] = ' '.join(
            tokens[superspan['st']:superspan['ed']])
    return superspan_list

In [11]:
data_dir = 'data'
input_file_name = 'arxiv_abstracts_10000.txt'
extensionless_input_file_name = input_file_name.split('.')[0]
input_file_path = f'{data_dir}/{input_file_name}'

output_file = f'{data_dir}/{extensionless_input_file_name}_superspan_sequence.json'

autophrase_file = f'{data_dir}/{extensionless_input_file_name}_autophrase.json'

log_out_list = f'{data_dir}/{extensionless_input_file_name}_outListFile.json'
log_source2freq_counter = f'{data_dir}/{extensionless_input_file_name}_source2freq_counter.bin'


texts = read_text(input_file_path)

data_method_list = []

if os.path.exists(autophrase_file):
    data_method_list.append((read_span_json(autophrase_file), 'autophrase'))

cutoff = min([len(l) for l, name in data_method_list])
data_method_list = [(data[:cutoff], method) for data, method in data_method_list]
texts = texts[:cutoff]
data_list, method_names = list(zip(*data_method_list))
new_data = merge_span_data(data_list, method_names)

source2freq_counter = calc_phrase_freq(new_data)
pickle.dump(source2freq_counter, open(log_source2freq_counter, 'wb'))

new_data = filter_span_data_by_grammar(new_data, texts)

with open(log_out_list, 'w') as fout_list, open(output_file, 'w') as fout:
    for doc_ind in tqdm(range(cutoff)):
        text = texts[doc_ind].strip()
        if text:
            tokens = text.split(' ')
        else:
            tokens = []

        superspan_list = generate_superspan(new_data[doc_ind], tokens)

        fout_list.write(json.dumps(superspan_list))
        fout_list.write('\n')

        superspan_sequence = generate_sequence(superspan_list, tokens)

        superspan_sequence = validate_compress_superspan_sequence(superspan_sequence, tokens)

        fout.write(json.dumps(superspan_sequence))
        fout.write('\n')

  0%|          | 0/10000 [00:00<?, ?it/s]

[10000]





  0%|          | 8/10000 [00:00<04:47, 34.75it/s]

  0%|          | 17/10000 [00:00<04:31, 36.77it/s]

  0%|          | 25/10000 [00:00<04:42, 35.29it/s]

...

100%|█████████▉| 9983/10000 [03:51<00:00, 39.50it/s]

100%|█████████▉| 9988/10000 [03:51<00:00, 40.70it/s]

100%|█████████▉| 9998/10000 [03:51<00:00, 37.07it/s]

  7%|▋         | 667/10000 [00:00<00:02, 3343.41it/s]

100%|██████████| 10000/10000 [00:02<00:00, 3391.68it/s]
